In [2]:
from xml.dom import minidom
import simplejson as json
def parse_element(element):
    dict_data = dict()
    if element.nodeType == element.TEXT_NODE:
        if element.data != " ":
            dict_data['data'] = element.data
    if element.nodeType not in [element.TEXT_NODE, element.DOCUMENT_NODE, 
                                element.DOCUMENT_TYPE_NODE]:
        for item in element.attributes.items():
            dict_data[item[0]] = item[1]
    if element.nodeType not in [element.TEXT_NODE, element.DOCUMENT_TYPE_NODE]:
        for child in element.childNodes:
            child_name, child_dict = parse_element(child)
            if child_name in dict_data:
                try:
                    dict_data[child_name].append(child_dict)
                except AttributeError:
                    dict_data[child_name] = [dict_data[child_name], child_dict]
            else:
                dict_data[child_name] = child_dict 
    return element.nodeName, dict_data

if __name__ == '__main__':
    dom = minidom.parse('cqa_task3.xml')
    print type(dom)
    f = open('cqa_task3.json', 'w')
    f.write(json.dumps(parse_element(dom), sort_keys=True, indent=4))
    f.close()

<type 'instance'>


In [84]:
import pandas as pd

data1 = pd.read_json('cqa_task3.json')
data1.head()
df31 = pd.io.json.json_normalize(data1[0][1]['xml'][1]['Thread'])
df41 = pd.concat([pd.DataFrame(pd.io.json.json_normalize(x)) for x in df31['RelComment']],ignore_index=True)
df41['THREAD_SEQUENCE'] = [ '_'.join(i.split('_')[:2]) for i in df41['RELC_ID']]
df51 = pd.concat([df31.set_index('THREAD_SEQUENCE'),df41.set_index('THREAD_SEQUENCE')], axis=1, join='inner').reset_index()
df51.head()
rem_cols = ['RelQuestion.#text' , 'RelComment', 'RelCBody.#text.data' , 'RelQuestion.RelQBody.#text.data' , 
            'RelQuestion.RelQBody.#text.data', '#text']
[ df51.drop(i, axis=1, inplace=True) for i in rem_cols  ]
df51.head()
#df51[ df51['RelQuestion.RelQSubject.#text.data'] == 'Best Bank.' ]

,THREAD_SEQUENCE,RelQuestion.RELQ_CATEGORY,RelQuestion.RELQ_DATE,RelQuestion.RELQ_ID,RelQuestion.RELQ_USERID,RelQuestion.RELQ_USERNAME,RelQuestion.RelQClean.#text.data,RelQuestion.RelQSubject.#text.data,RELC_DATE,RELC_ID,RELC_RELEVANCE2RELQ,RELC_USERID,RELC_USERNAME,RelCClean.#text.data
0,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-07-31 06:46:39,Q268_R16_C1,Bad,U65,Molten Metal,banks are using us ... Talk to those who had t...
1,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-07-31 08:10:53,Q268_R16_C2,Bad,U956,Rip Cord,In Qatar that is like saying which is the best...
2,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-07-31 10:37:03,Q268_R16_C3,Bad,U5152,arman1arzoo,I'm surprised to see such feedbacks on Qatar b...
3,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-08-01 11:38:21,Q268_R16_C4,Good,U5153,westernindoha,Well Arman; nothing is wrong here with banks; ...
4,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-08-01 11:45:49,Q268_R16_C5,Good,U492,happygolucky,With QNB for last 4 years plus...no issues...g...


In [100]:
df_new = df51

In [101]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

def preprocess_words(x):
    example_sent = x
    stop_words = set(stopwords.words('english'))
    tokenizer = RegexpTokenizer(r'\w+')
    word_tokens = tokenizer.tokenize(example_sent)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)


In [102]:
import re, math
from collections import Counter

WORD = re.compile(r'\w+')

def text_to_vector(text):
     words = WORD.findall(text)
     return Counter(words)

def get_cosine(text1, text2):
     vec1 = text_to_vector(text1)
     vec2 = text_to_vector(text2)
     intersection = set(vec1.keys()) & set(vec2.keys())
     numerator = sum([vec1[x] * vec2[x] for x in intersection])

     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)
     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator
    
df_new['tokenized_question'] = df_new['RelQuestion.RelQClean.#text.data'].apply(lambda row: preprocess_words(str(''.join([c for c in row if ord(c) < 128]))))
df_new['tokenized_comment'] = df_new['RelCClean.#text.data'].apply(lambda row: preprocess_words(str(''.join([c for c in row if ord(c) < 128]))))
df_new['cosine_similarity_q2c'] = df_new.apply(lambda row : get_cosine( str(row['tokenized_question']),str(row['tokenized_comment']) ),axis =1)
df_new['pos_tag_ques'] = df_new['RelQuestion.RelQClean.#text.data'].apply(lambda x: nltk.pos_tag(x.split()) )
df_new['pos_tag_ans'] = df_new['RelCClean.#text.data'].apply(lambda x: nltk.pos_tag(x.split()) )


In [116]:
df_new.head()
each_df = df_new[ df_new['THREAD_SEQUENCE'] == 'Q268_R16']
#each_df[['RELC_RELEVANCE2RELQ','cosine_similarity_q2c']]
each_df

,THREAD_SEQUENCE,RelQuestion.RELQ_CATEGORY,RelQuestion.RELQ_DATE,RelQuestion.RELQ_ID,RelQuestion.RELQ_USERID,RelQuestion.RELQ_USERNAME,RelQuestion.RelQClean.#text.data,RelQuestion.RelQSubject.#text.data,RELC_DATE,RELC_ID,RELC_RELEVANCE2RELQ,RELC_USERID,RELC_USERNAME,RelCClean.#text.data,tokenized_question,tokenized_comment,cosine_similarity_q2c,pos_tag_ques,pos_tag_ans
0,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-07-31 06:46:39,Q268_R16_C1,Bad,U65,Molten Metal,banks are using us ... Talk to those who had t...,Best Bank Hi ti QL What bank using Are using b...,banks using us Talk taken credit card loan know,0.157135,"[(Best, NNP), (Bank., NNP), (//, NNP), (Hi, NN...","[(banks, NNS), (are, VBP), (using, VBG), (us, ..."
1,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-07-31 08:10:53,Q268_R16_C2,Bad,U956,Rip Cord,In Qatar that is like saying which is the best...,Best Bank Hi ti QL What bank using Are using b...,In Qatar like saying best STD,0.000000,"[(Best, NNP), (Bank., NNP), (//, NNP), (Hi, NN...","[(In, IN), (Qatar, NNP), (that, WDT), (is, VBZ..."
2,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-07-31 10:37:03,Q268_R16_C3,Bad,U5152,arman1arzoo,I'm surprised to see such feedbacks on Qatar b...,Best Bank Hi ti QL What bank using Are using b...,I surprised see feedbacks Qatar banks Is serio...,0.169334,"[(Best, NNP), (Bank., NNP), (//, NNP), (Hi, NN...","[(I'm, NNP), (surprised, VBD), (to, TO), (see,..."
3,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-08-01 11:38:21,Q268_R16_C4,Good,U5153,westernindoha,Well Arman; nothing is wrong here with banks; ...,Best Bank Hi ti QL What bank using Are using b...,Well Arman nothing wrong banks I feel par UAE ...,0.091574,"[(Best, NNP), (Bank., NNP), (//, NNP), (Hi, NN...","[(Well, RB), (Arman;, NNP), (nothing, NN), (is..."
4,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-08-01 11:45:49,Q268_R16_C5,Good,U492,happygolucky,With QNB for last 4 years plus...no issues...g...,Best Bank Hi ti QL What bank using Are using b...,With QNB last 4 years plus issues great servic...,0.000000,"[(Best, NNP), (Bank., NNP), (//, NNP), (Hi, NN...","[(With, IN), (QNB, NNP), (for, IN), (last, JJ)..."
5,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-08-02 06:38:26,Q268_R16_C6,PotentiallyUseful,U5151,shehabi,WesternInDoha; that's the information that I a...,Best Bank Hi ti QL What bank using Are using b...,WesternInDoha information I looking answer que...,0.000000,"[(Best, NNP), (Bank., NNP), (//, NNP), (Hi, NN...","[(WesternInDoha;, NNP), (that's, VBZ), (the, D..."
6,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-08-02 06:46:37,Q268_R16_C7,PotentiallyUseful,U5151,shehabi,MoltenMetal; it depend how you are looking on ...,Best Bank Hi ti QL What bank using Are using b...,MoltenMetal depend looking subject matter poin...,0.000000,"[(Best, NNP), (Bank., NNP), (//, NNP), (Hi, NN...","[(MoltenMetal;, NN), (it, PRP), (depend, VB), ..."
7,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-08-02 06:51:51,Q268_R16_C8,Bad,U5151,shehabi,That's new way of description for the Bank's.....,Best Bank Hi ti QL What bank using Are using b...,That new way description Bank I planing let co...,0.162221,"[(Best, NNP), (Bank., NNP), (//, NNP), (Hi, NN...","[(That's, NNP), (new, JJ), (way, NN), (of, IN)..."
8,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268

In [ ]:
train_set = df
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [123]:
col_list = list(each_df.columns)

In [124]:
col_list.remove(u'RELC_RELEVANCE2RELQ')

In [129]:
import plotly.plotly as py
import plotly.figure_factory as ff

df = [dict(Task="Job A", Start='2009-01-01', Finish='2009-02-28', Complete=10),
      dict(Task="Job B", Start='2008-12-05', Finish='2009-04-15', Complete=60),
      dict(Task="Job C", Start='2009-02-20', Finish='2009-05-30', Complete=95)]

fig = ff.create_gantt(df, colors='Viridis', index_col='Complete', show_colorbar=True)
py.iplot(fig, filename='gantt-numeric-variable', world_readable=True)

/Users/srimi/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning:

Matplotlib is building the font cache using fc-list. This may take a moment.



Aw, snap! We don't have an account for ''. Want to try again? You can authenticate with your email address or username. Sign in is not case sensitive.

Don't have an account? plot.ly

Questions? support@plot.ly


PlotlyError: Because you didn't supply a 'file_id' in the call, we're assuming you're trying to snag a figure from a url. You supplied the url, '', we expected it to start with 'https://plot.ly'.
Run help on this function for more information.

In [131]:

 
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import matplotlib.dates
from matplotlib.dates import MONTHLY, DateFormatter, rrulewrapper, RRuleLocator
 
from pylab import *
 
def create_date(month,year):

date = dt.datetime(int(year), int(month), 1)
mdate = matplotlib.dates.date2num(date)
 
return mdate
 
# Data
 
pos = arange(0.5,5.5,0.5)
 
ylabels = []
ylabels.append('Hardware Design & Review')
ylabels.append('Hardware Construction')
ylabels.append('Integrate and Test Laser Source')
ylabels.append('Objective #1')
ylabels.append('Objective #2')
ylabels.append('Present at ASMS')
ylabels.append('Present Data at Gordon Conference')
ylabels.append('Manuscripts and Final Report')
 
effort = []
effort.append([0.2, 1.0])
effort.append([0.2, 1.0])
effort.append([0.2, 1.0])
effort.append([0.3, 0.75])
effort.append([0.25, 0.75])
effort.append([0.3, 0.75])
effort.append([0.5, 0.5])
effort.append([0.7, 0.4])
 
customDates = []
customDates.append([create_date(5,2014),create_date(6,2014)])
customDates.append([create_date(6,2014),create_date(8,2014),create_date(8,2014)])
customDates.append([create_date(7,2014),create_date(9,2014),create_date(9,2014)])
customDates.append([create_date(10,2014),create_date(3,2015),create_date(3,2015)])
customDates.append([create_date(2,2015),create_date(6,2015),create_date(6,2015)])
customDates.append([create_date(5,2015),create_date(6,2015),create_date(6,2015)])
customDates.append([create_date(6,2015),create_date(7,2015),create_date(7,2015)])
customDates.append([create_date(4,2015),create_date(8,2015),create_date(8,2015)])
 
task_dates = {}
for i,task in enumerate(ylabels):
task_dates[task] = customDates[i]
# task_dates['Climatology'] = [create_date(5,2014),create_date(6,2014),create_date(10,2013)]
# task_dates['Structure'] = [create_date(10,2013),create_date(3,2014),create_date(5,2014)]
# task_dates['Impacts'] = [create_date(5,2014),create_date(12,2014),create_date(2,2015)]
# task_dates['Thesis'] = [create_date(2,2015),create_date(5,2015)]
 
# Initialise plot
 
fig = plt.figure()
# ax = fig.add_axes([0.15,0.2,0.75,0.3]) #[left,bottom,width,height]
ax = fig.add_subplot(111)
 
# Plot the data
 
start_date,end_date = task_dates[ylabels[0]]
ax.barh(0.5, end_date - start_date, left=start_date, height=0.3, align='center', color='blue', alpha = 0.75)
ax.barh(0.45, (end_date - start_date)*effort[0][0], left=start_date, height=0.1, align='center', color='red', alpha = 0.75, label = "PI Effort")
ax.barh(0.55, (end_date - start_date)*effort[0][1], left=start_date, height=0.1, align='center', color='yellow', alpha = 0.75, label = "Student Effort")
for i in range(0,len(ylabels)-1):
labels = ['Analysis','Reporting'] if i == 1 else [None,None]
start_date,mid_date,end_date = task_dates[ylabels[i+1]]
piEffort, studentEffort = effort[i+1]
ax.barh((i*0.5)+1.0, mid_date - start_date, left=start_date, height=0.3, align='center', color='blue', alpha = 0.75)
ax.barh((i*0.5)+1.0-0.05, (mid_date - start_date)*piEffort, left=start_date, height=0.1, align='center', color='red', alpha = 0.75)
ax.barh((i*0.5)+1.0+0.05, (mid_date - start_date)*studentEffort, left=start_date, height=0.1, align='center', color='yellow', alpha = 0.75)
# ax.barh((i*0.5)+1.0, end_date - mid_date, left=mid_date, height=0.3, align='center',label=labels[1], color='yellow')
 
# Format the y-axis
 
locsy, labelsy = yticks(pos,ylabels)
plt.setp(labelsy, fontsize = 14)
 
# Format the x-axis
 
ax.axis('tight')
ax.set_ylim(ymin = -0.1, ymax = 4.5)
ax.grid(color = 'g', linestyle = ':')
 
ax.xaxis_date() #Tell matplotlib that these are dates...
 
rule = rrulewrapper(MONTHLY, interval=1)
loc = RRuleLocator(rule)
formatter = DateFormatter("%b '%y")
 
ax.xaxis.set_major_locator(loc)
ax.xaxis.set_major_formatter(formatter)
labelsx = ax.get_xticklabels()
plt.setp(labelsx, rotation=30, fontsize=12)
 
# Format the legend
 
font = font_manager.FontProperties(size='small')
ax.legend(loc=1,prop=font)
 
# Finish up
ax.invert_yaxis()
fig.autofmt_xdate()
#plt.savefig('gantt.svg')
plt.show()

IndentationError: expected an indented block (<ipython-input-131-6ddd16d0effb>, line 14)

In [135]:

import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import matplotlib.dates
from matplotlib.dates import WEEKLY,MONTHLY, DateFormatter, rrulewrapper, RRuleLocator 
import numpy as np
  
 
def _create_date(datetxt):
    """Creates the date"""
    day,month,year=datetxt.split('-')
    date = dt.datetime(int(year), int(month), int(day))
    mdate = matplotlib.dates.date2num(date) 
    return mdate
 
def CreateGanttChart(fname):
    """
        Create gantt charts with matplotlib
        Give file name.
    """ 
    ylabels = []
    customDates = []
    try:
        textlist=open(fname).readlines()
    except:
        return
#
    for tx in textlist:
        if not tx.startswith('#'):
            ylabel,startdate,enddate=tx.split(',')
            ylabels.append(ylabel.replace('\n',''))
            customDates.append([_create_date(startdate.replace('\n','')),_create_date(enddate.replace('\n',''))])
             
    ilen=len(ylabels)
    pos = np.arange(0.5,ilen*0.5+0.5,0.5)
    task_dates = {}
    for i,task in enumerate(ylabels):
        task_dates[task] = customDates[i]
    fig = plt.figure(figsize=(20,8))
    ax = fig.add_subplot(111)
    for i in range(len(ylabels)):
         start_date,end_date = task_dates[ylabels[i]]
         ax.barh((i*0.5)+0.5, end_date - start_date, left=start_date, height=0.3, align='center', edgecolor='lightgreen', color='orange', alpha = 0.8)
    locsy, labelsy = plt.yticks(pos,ylabels)
    plt.setp(labelsy, fontsize = 14)
#    ax.axis('tight')
    ax.set_ylim(ymin = -0.1, ymax = ilen*0.5+0.5)
    ax.grid(color = 'g', linestyle = ':')
    ax.xaxis_date()
    rule = rrulewrapper(WEEKLY, interval=1)
    loc = RRuleLocator(rule)
    #formatter = DateFormatter("%d-%b '%y")
    formatter = DateFormatter("%d-%b")
  
    ax.xaxis.set_major_locator(loc)
    ax.xaxis.set_major_formatter(formatter)
    labelsx = ax.get_xticklabels()
    plt.setp(labelsx, rotation=30, fontsize=10)
 
    font = font_manager.FontProperties(size='small')
    ax.legend(loc=1,prop=font)
 
    ax.invert_yaxis()
    fig.autofmt_xdate()
    plt.savefig('gantt.svg')
    plt.show()
 

fname=r"gant_data.txt"
CreateGanttChart(fname)